# A3. Graph API 差分同期

## 概要

このノートブックでは、Microsoft Graph の **Delta Query API** を使って、SharePoint の変更を効率的に検出し、Azure AI Search インデックスを更新します。

### Delta Query とは

Delta Query は、前回の同期以降に変更されたアイテム (追加/更新/削除) のみを取得できる API です。

**メリット**:
- ⚡ 高速: 変更されたファイルのみを処理
- 💰 コスト削減: API 呼び出し回数と処理時間を削減
- 🔄 リアルタイム性: 定期実行で常に最新状態を維持

### 処理フロー

1. **Delta Token の取得**: 前回同期時のトークンを読み込み
2. **変更の取得**: Delta API で変更されたアイテムを取得
3. **変更の分類**: 追加/更新/削除を判定
4. **インデックス更新**: 
   - 追加/更新: A2 と同じ処理でドキュメント生成→アップロード
   - 削除: インデックスから該当ドキュメントを削除
5. **Token の保存**: 次回同期用に Delta Token を保存

### 所要時間

- 初回実行: 5〜10分 (Delta Token の初期化)
- 2回目以降: 数秒〜数分 (変更量に依存)

## 1. 環境の初期化

In [ ]:
import os
import json
import msal
import requests
import time
from pathlib import Path
from dotenv import load_dotenv
from typing import Dict, List, Optional, Tuple
from datetime import datetime
from tqdm.auto import tqdm

# Azure Search
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient

# ユーティリティ
from utils import (
    TextExtractor,
    TextChunker,
    create_document_id,
    extract_acl_from_permissions
)

# 環境変数の読み込み
load_dotenv(Path("..") / ".env")

# 設定の読み込み
config_path = Path("..") / "config.json"
if config_path.exists():
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)
    print("✅ 設定を読み込みました")
else:
    print("❌ config.json が見つかりません")
    raise FileNotFoundError("config.json not found")

# Delta Token ファイルのパス
delta_token_file = Path("..") / os.getenv("DELTA_TOKEN_FILE", ".delta_token.json")

print(f"\nサイト: {config['site_info']['name']}")
print(f"Delta Token ファイル: {delta_token_file}")

## 2. Graph API クライアントの準備

In [ ]:
class GraphAPIClient:
    def __init__(self, tenant_id: str, client_id: str, client_secret: str):
        self.tenant_id = tenant_id
        self.client_id = client_id
        self.authority = f"https://login.microsoftonline.com/{tenant_id}"
        self.base_url = "https://graph.microsoft.com/v1.0"
        
        self.app = msal.ConfidentialClientApplication(
            client_id,
            authority=self.authority,
            client_credential=client_secret
        )
        self.token = None
    
    def get_token(self) -> str:
        result = self.app.acquire_token_for_client(
            scopes=["https://graph.microsoft.com/.default"]
        )
        if "access_token" in result:
            self.token = result["access_token"]
            return self.token
        else:
            raise Exception(f"Token error: {result.get('error_description')}")
    
    def get_headers(self) -> Dict[str, str]:
        if not self.token:
            self.get_token()
        return {
            "Authorization": f"Bearer {self.token}",
            "Content-Type": "application/json"
        }
    
    def get(self, endpoint: str, params: Optional[Dict] = None) -> Dict:
        url = f"{self.base_url}{endpoint}"
        response = requests.get(url, headers=self.get_headers(), params=params)
        
        # リトライロジック
        retry_count = 0
        while response.status_code == 429 and retry_count < 3:
            retry_after = int(response.headers.get("Retry-After", 5))
            print(f"⚠️  Rate limited. Waiting {retry_after} seconds...")
            time.sleep(retry_after)
            response = requests.get(url, headers=self.get_headers(), params=params)
            retry_count += 1
        
        response.raise_for_status()
        return response.json()
    
    def get_with_url(self, full_url: str) -> Dict:
        """フルURLで直接リクエスト (Delta API の nextLink/deltaLink 用)"""
        response = requests.get(full_url, headers=self.get_headers())
        
        retry_count = 0
        while response.status_code == 429 and retry_count < 3:
            retry_after = int(response.headers.get("Retry-After", 5))
            print(f"⚠️  Rate limited. Waiting {retry_after} seconds...")
            time.sleep(retry_after)
            response = requests.get(full_url, headers=self.get_headers())
            retry_count += 1
        
        response.raise_for_status()
        return response.json()

# クライアント作成
graph_client = GraphAPIClient(
    tenant_id=os.getenv("GRAPH_TENANT_ID"),
    client_id=os.getenv("GRAPH_CLIENT_ID"),
    client_secret=os.getenv("GRAPH_CLIENT_SECRET")
)

print("✅ Graph API クライアントを初期化しました")

## 3. Delta Token の管理

In [ ]:
def load_delta_tokens() -> Dict[str, str]:
    """
    Delta Token を読み込む
    ドライブごとに Token を管理
    """
    if delta_token_file.exists():
        with open(delta_token_file, "r", encoding="utf-8") as f:
            tokens = json.load(f)
        print(f"✅ Delta Token を読み込みました ({len(tokens)} ドライブ)")
        return tokens
    else:
        print("ℹ️  Delta Token ファイルが存在しません (初回実行)")
        return {}

def save_delta_tokens(tokens: Dict[str, str]):
    """
    Delta Token を保存
    """
    with open(delta_token_file, "w", encoding="utf-8") as f:
        json.dump(tokens, f, indent=2, ensure_ascii=False)
    print(f"✅ Delta Token を保存しました: {delta_token_file}")

# 既存の Token を読み込み
delta_tokens = load_delta_tokens()

print("\n現在の Token 状態:")
for drive in config["target_drives"]:
    drive_id = drive["id"]
    has_token = drive_id in delta_tokens
    status = "✓" if has_token else "✗"
    print(f"  {status} {drive['name']}: {'Token あり' if has_token else 'Token なし (初回同期が必要)'}")

## 4. Delta API による変更の取得

In [ ]:
def get_delta_changes(
    graph_client: GraphAPIClient,
    drive_id: str,
    delta_link: Optional[str] = None
) -> Tuple[List[Dict], str]:
    """
    Delta API で変更を取得
    
    Returns:
        (changes, new_delta_link)
    """
    all_changes = []
    
    # 初回またはdeltaLinkがある場合
    if delta_link:
        # 既存の deltaLink を使用
        url = delta_link
    else:
        # 初回: delta エンドポイントを使用
        url = f"{graph_client.base_url}/drives/{drive_id}/root/delta"
    
    while url:
        if url.startswith("http"):
            response = graph_client.get_with_url(url)
        else:
            response = graph_client.get(url)
        
        # 変更を追加
        all_changes.extend(response.get("value", []))
        
        # 次のページまたは deltaLink
        url = response.get("@odata.nextLink")  # まだ続きがある場合
        if not url:
            # すべて取得完了: deltaLink を保存
            new_delta_link = response.get("@odata.deltaLink")
            break
    
    return all_changes, new_delta_link

def classify_changes(
    changes: List[Dict],
    supported_extensions: List[str]
) -> Tuple[List[Dict], List[Dict], List[str]]:
    """
    変更を分類
    
    Returns:
        (added_files, updated_files, deleted_ids)
    """
    added = []
    updated = []
    deleted = []
    
    for item in changes:
        # 削除されたアイテム
        if "deleted" in item or item.get("@removed"):
            deleted.append(item["id"])
            continue
        
        # フォルダはスキップ
        if "folder" in item:
            continue
        
        # ファイル以外はスキップ
        if "file" not in item:
            continue
        
        # サポート対象の拡張子かチェック
        name = item.get("name", "")
        ext = Path(name).suffix.lower()
        
        if ext not in supported_extensions:
            continue
        
        # ファイル情報を構造化
        file_info = {
            "id": item["id"],
            "name": name,
            "path": item.get("parentReference", {}).get("path", "") + "/" + name,
            "extension": ext,
            "size": item.get("size", 0),
            "webUrl": item.get("webUrl", ""),
            "lastModified": item.get("lastModifiedDateTime", ""),
            "createdBy": item.get("createdBy", {}).get("user", {}).get("displayName", ""),
            "modifiedBy": item.get("lastModifiedBy", {}).get("user", {}).get("displayName", ""),
        }
        
        # 新規か更新かを判定 (createdDateTime と lastModifiedDateTime で判定)
        created = item.get("createdDateTime", "")
        modified = item.get("lastModifiedDateTime", "")
        
        # 簡易判定: 作成日時と更新日時が近ければ新規
        if created and modified:
            created_dt = datetime.fromisoformat(created.replace("Z", "+00:00"))
            modified_dt = datetime.fromisoformat(modified.replace("Z", "+00:00"))
            diff_seconds = (modified_dt - created_dt).total_seconds()
            
            if diff_seconds < 60:  # 1分以内なら新規
                added.append(file_info)
            else:
                updated.append(file_info)
        else:
            # 判定できない場合は更新扱い
            updated.append(file_info)
    
    return added, updated, deleted

print("✅ Delta API 関数を定義しました")

## 5. 変更の取得と分類

In [ ]:
# 全ドライブの変更を取得
all_added = []
all_updated = []
all_deleted = []
new_delta_tokens = {}

print("\n変更を取得しています...\n")
print("=" * 60)

for drive in config["target_drives"]:
    drive_id = drive["id"]
    drive_name = drive["name"]
    
    print(f"\n📁 {drive_name}")
    
    # 既存の Delta Token を取得
    delta_link = delta_tokens.get(drive_id)
    
    if delta_link:
        print("   既存の Delta Token を使用して差分を取得...")
    else:
        print("   初回実行: すべてのアイテムを取得...")
    
    try:
        # Delta API 呼び出し
        changes, new_delta_link = get_delta_changes(
            graph_client,
            drive_id,
            delta_link
        )
        
        print(f"   取得した変更: {len(changes)} 件")
        
        # 変更を分類
        added, updated, deleted = classify_changes(
            changes,
            config["supported_extensions"]
        )
        
        # ドライブ情報を追加
        for f in added:
            f["drive_id"] = drive_id
            f["drive_name"] = drive_name
        for f in updated:
            f["drive_id"] = drive_id
            f["drive_name"] = drive_name
        
        all_added.extend(added)
        all_updated.extend(updated)
        all_deleted.extend(deleted)
        
        # 新しい Delta Token を保存
        new_delta_tokens[drive_id] = new_delta_link
        
        print(f"   ✅ 追加: {len(added)} 件")
        print(f"   ✅ 更新: {len(updated)} 件")
        print(f"   ✅ 削除: {len(deleted)} 件")
    
    except Exception as e:
        print(f"   ❌ エラー: {str(e)[:100]}")
        continue

print("\n" + "=" * 60)
print("📊 変更サマリー")
print("=" * 60)
print(f"追加ファイル: {len(all_added)} 件")
print(f"更新ファイル: {len(all_updated)} 件")
print(f"削除ファイル: {len(all_deleted)} 件")
print(f"合計: {len(all_added) + len(all_updated) + len(all_deleted)} 件")
print("=" * 60)

## 6. Azure AI Search の準備

In [ ]:
# Search クライアント
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")

search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_key)
)

# Azure OpenAI (オプション)
use_vector_search = bool(os.getenv("AZURE_OPENAI_ENDPOINT"))
openai_client = None
embedding_deployment = None

if use_vector_search:
    try:
        from openai import AzureOpenAI
        
        openai_client = AzureOpenAI(
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")
        )
        embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
        print(f"✅ ベクター検索: 有効 ({embedding_deployment})")
    except:
        use_vector_search = False
        print("⚠️  ベクター検索: 無効")
else:
    print("⚠️  ベクター検索: 無効")

print(f"\nインデックス: {index_name}")

## 7. ファイル処理関数 (A2 から再利用)

In [ ]:
def process_file(
    graph_client: GraphAPIClient,
    file_info: Dict,
    site_info: Dict,
    chunker: TextChunker,
    openai_client = None,
    embedding_deployment: str = None
) -> List[Dict]:
    """
    1つのファイルを処理してドキュメントリストを返す
    """
    try:
        # ファイルコンテンツのダウンロード
        download_url = f"/drives/{file_info['drive_id']}/items/{file_info['id']}/content"
        content_response = requests.get(
            f"{graph_client.base_url}{download_url}",
            headers=graph_client.get_headers(),
            allow_redirects=True
        )
        content_response.raise_for_status()
        content_bytes = content_response.content
        
        # テキスト抽出
        text = TextExtractor.extract(content_bytes, file_info["extension"])
        
        if not text or len(text.strip()) < 10:
            return []  # 空のファイルはスキップ
        
        # ACL取得 (本番環境実装)
        acl_users = []
        acl_groups = []
        
        try:
            # Permissions APIを呼び出してアクセス権限を取得
            permissions_endpoint = f"/drives/{file_info['drive_id']}/items/{file_info['id']}/permissions"
            permissions_response = graph_client.get(permissions_endpoint)
            permissions = permissions_response.get("value", [])
            
            # ACLを抽出
            acl_users, acl_groups = extract_acl_from_permissions(permissions)
            
        except Exception as e:
            # Permission取得に失敗しても処理を続行
            print(f"⚠️  Permission取得エラー ({file_info['name']}): {str(e)[:80]}")
        
        # チャンク分割
        chunks = chunker.split_text(text, metadata=file_info)
        
        # ドキュメント生成
        documents = []
        for chunk in chunks:
            doc_id = create_document_id(
                site_info["id"],
                file_info["drive_id"],
                file_info["id"],
                chunk.chunk_index
            )
            
            doc = {
                "id": doc_id,
                "title": file_info["name"],
                "content": chunk.text,
                "url": file_info["webUrl"],
                "path": file_info["path"],
                "site": site_info["name"],
                "library": file_info["drive_name"],
                "contentType": file_info["extension"][1:] if file_info["extension"] else "unknown",
                "fileExtension": file_info["extension"],
                "lastModified": file_info["lastModified"],
                "createdBy": file_info["createdBy"],
                "modifiedBy": file_info["modifiedBy"],
                "size": file_info["size"],
                "chunkIndex": chunk.chunk_index,
                "aclUsers": acl_users,
                "aclGroups": acl_groups,
            }
            
            # 埋め込み生成 (オプション)
            if openai_client and embedding_deployment:
                try:
                    response = openai_client.embeddings.create(
                        input=chunk.text[:8000],  # トークン制限対策
                        model=embedding_deployment
                    )
                    doc["contentVector"] = response.data[0].embedding
                except Exception as e:
                    print(f"⚠️  埋め込み生成エラー ({file_info['name']}): {str(e)[:50]}")
            
            documents.append(doc)
        
        return documents
    
    except Exception as e:
        print(f"❌ ファイル処理エラー ({file_info['name']}): {str(e)[:100]}")
        return []

# チャンカーの準備
chunker = TextChunker(
    chunk_size=config["chunk_size"],
    chunk_overlap=config["chunk_overlap"]
)

print("✅ ファイル処理の準備が完了しました")
print(f"   チャンクサイズ: {config['chunk_size']} 文字")
print(f"   オーバーラップ: {config['chunk_overlap']} 文字")

## 8. 追加・更新ファイルの処理

In [ ]:
# 追加・更新ファイルをまとめて処理
files_to_process = all_added + all_updated

if files_to_process:
    print(f"\n{len(files_to_process)} 件のファイルを処理します...\n")
    print("=" * 60)
    
    all_documents = []
    failed_files = []
    
    for file_info in tqdm(files_to_process, desc="処理中"):
        docs = process_file(
            graph_client,
            file_info,
            config["site_info"],
            chunker,
            openai_client,
            embedding_deployment
        )
        
        if docs:
            all_documents.extend(docs)
        else:
            failed_files.append(file_info["name"])
        
        # API制限対策
        if use_vector_search:
            time.sleep(0.1)
    
    print("\n" + "=" * 60)
    print(f"✅ 処理完了: {len(all_documents)} ドキュメント生成")
    if failed_files:
        print(f"⚠️  失敗: {len(failed_files)} ファイル")
    print("=" * 60)
    
    # アップロード
    if all_documents:
        print("\nAzure AI Search にアップロードしています...")
        
        batch_size = 100
        total_uploaded = 0
        total_failed = 0
        
        for i in tqdm(range(0, len(all_documents), batch_size), desc="アップロード中"):
            batch = all_documents[i:i + batch_size]
            
            try:
                result = search_client.upload_documents(documents=batch)
                
                for res in result:
                    if res.succeeded:
                        total_uploaded += 1
                    else:
                        total_failed += 1
            
            except Exception as e:
                total_failed += len(batch)
                print(f"❌ アップロードエラー: {str(e)[:100]}")
            
            time.sleep(0.5)
        
        print(f"\n✅ アップロード完了: {total_uploaded} / {len(all_documents)} ドキュメント")
        if total_failed > 0:
            print(f"⚠️  失敗: {total_failed} ドキュメント")
else:
    print("\nℹ️  追加・更新するファイルはありません")

## 9. 削除ファイルの処理

In [ ]:
if all_deleted:
    print(f"\n{len(all_deleted)} 件の削除ファイルを処理します...\n")
    print("=" * 60)
    
    # 削除対象のドキュメント ID を生成
    # SharePoint のファイル ID をもとに、インデックス内の関連ドキュメントを検索して削除
    deleted_count = 0
    
    for file_id in tqdm(all_deleted, desc="削除中"):
        try:
            # ファイル ID を含むドキュメントを検索
            # ドキュメント ID のパターン: {site_id}_{drive_id}_{file_id}_{chunk_index}
            # ワイルドカードは使えないので、filter で検索
            
            # 簡易的な方法: id にファイル ID が含まれるものを検索
            # (本番環境では、ファイル ID を別フィールドで管理することを推奨)
            search_results = search_client.search(
                search_text="*",
                filter=f"search.in(id, '{file_id}', '_')",  # 部分一致検索の代替
                select=["id"],
                top=1000
            )
            
            docs_to_delete = []
            for result in search_results:
                # ID にファイル ID が含まれているかチェック
                if file_id in result["id"]:
                    docs_to_delete.append({"id": result["id"]})
            
            if docs_to_delete:
                # 削除実行
                delete_result = search_client.delete_documents(documents=docs_to_delete)
                deleted_count += len([r for r in delete_result if r.succeeded])
        
        except Exception as e:
            print(f"❌ 削除エラー ({file_id}): {str(e)[:100]}")
        
        time.sleep(0.1)
    
    print("\n" + "=" * 60)
    print(f"✅ 削除完了: {deleted_count} ドキュメント")
    print("=" * 60)
else:
    print("\nℹ️  削除するファイルはありません")

## 10. Delta Token の保存

In [ ]:
# 新しい Delta Token を保存
save_delta_tokens(new_delta_tokens)

print("\n" + "=" * 60)
print("✅ 差分同期が完了しました")
print("=" * 60)
print(f"\n処理サマリー:")
print(f"  追加: {len(all_added)} ファイル")
print(f"  更新: {len(all_updated)} ファイル")
print(f"  削除: {len(all_deleted)} ファイル")
print(f"\n次回実行時は、このタイミング以降の変更のみが処理されます")
print("=" * 60)

## 11. インデックス統計の確認

In [ ]:
# インデックス統計
time.sleep(3)

try:
    results = search_client.search(
        search_text="*",
        top=1,
        include_total_count=True
    )
    
    total_count = results.get_count()
    
    print("\n" + "=" * 60)
    print("📊 現在のインデックス統計")
    print("=" * 60)
    print(f"インデックス名: {index_name}")
    print(f"ドキュメント数: {total_count:,} 件")
    print(f"最終更新: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 60)
    
except Exception as e:
    print(f"⚠️  統計取得エラー: {e}")

## まとめ

このノートブックで以下を完了しました:

✅ Delta API による変更の検出  
✅ 追加・更新・削除の分類  
✅ インデックスの増分更新  
✅ Delta Token の保存  

### 次のステップ

**A4_query_and_acl_demo.ipynb** に進んで、検索機能と ACL フィルタリングを試します。

### 定期実行の設定

このノートブックを定期実行することで、常に最新のインデックスを維持できます。

**実行方法の例**:

1. **Azure Functions + Timer Trigger**: 
   - このノートブックのロジックを Python スクリプトに変換
   - 5分〜1時間ごとに実行

2. **GitHub Actions**:
   - Papermill で Notebook を実行
   - cron で定期実行

3. **Azure Data Factory**:
   - Notebook Activity で実行
   - スケジュール トリガーで定期実行

### パフォーマンスヒント

**初回実行**:
- すべてのファイルを取得するため時間がかかります
- ファイル数が多い場合は、ドライブごとに分割実行を推奨

**2回目以降**:
- Delta Token により変更のみを処理
- 通常は数秒〜数分で完了

**最適化のポイント**:
- 並列処理: `concurrent.futures.ThreadPoolExecutor`
- バッチサイズ: API の制限に合わせて調整
- エラーハンドリング: リトライロジックの強化

### トラブルシューティング

**Delta Token エラー**:
- Token が古すぎる場合は無効になります
- `.delta_token.json` を削除して再実行

**削除処理の改善**:
- 現在の実装は簡易版です
- 本番環境では、ファイル ID を専用フィールドで管理することを推奨
- インデックススキーマに `sourceFileId` フィールドを追加

**変更の検出漏れ**:
- Delta API は権限変更のみでもイベントを返します
- 必要に応じて、権限変更時の ACL 更新処理を追加